In [1]:
%load_ext sql

In [2]:
%sql sqlite:///council.db

In [3]:
%%sql
DROP TABLE IF EXISTS Researcher;
CREATE TABLE Researcher (
    emailAddress,
    firstName CHAR(15),
    lastName CHAR(15),
    Organization CHAR(10),
    PRIMARY KEY (emailAddress)
);

 * sqlite:///council.db
Done.
Done.


[]

In [4]:
%%sql
DROP TABLE IF EXISTS Grants;
CREATE TABLE Grants (
    callNumber INTEGER PRIMARY KEY AUTOINCREMENT,
    applicationDeadline DATE DEFAULT CURRENT_DATE,
    status TEXT CHECK (status IN ('open', 'closed'))
);

DROP TABLE IF EXISTS GrantDetails;
CREATE TABLE GrantDetails (
    callNumber INTEGER PRIMARY KEY,
    title TEXT NOT NULL CHECK (title <> ''),
    description TEXT DEFAULT '',
    area CHAR(15) DEFAULT '',
    FOREIGN KEY (callNumber) REFERENCES Grants(callNumber)
);

 * sqlite:///council.db
Done.
Done.
Done.
Done.


[]

In [5]:
%%sql
DROP TABLE IF EXISTS ProposalDetails;
CREATE TABLE ProposalDetails (
    proposalNumber INTEGER PRIMARY KEY AUTOINCREMENT,
    requestedAmount INTEGER,
    applicationStatus TEXT NOT NULL CHECK (applicationStatus IN ('submitted', 'awarded', 'not awarded' )),
    awardedAmount INTEGER,
    date DATE,
    CHECK ((applicationStatus = 'awarded' AND awardedAmount IS NOT NULL) OR (applicationStatus <> 'awarded' AND awardedAmount IS NULL))
);

DROP TABLE IF EXISTS GrantProposals;
CREATE TABLE GrantProposals (
    proposalNumber INTEGER,
    callNumber INTEGER,
    principalInvestigator TEXT NOT NULL,
    collaborators TEXT,
    FOREIGN KEY (proposalNumber) REFERENCES ProposalDetails(proposalNumber),
    FOREIGN KEY (callNumber) REFERENCES Researcher(emailAddress),
    FOREIGN KEY (principalInvestigator) REFERENCES Researcher(emailAddress),
    FOREIGN KEY (collaborators) REFERENCES Researcher(emailAddress),
    PRIMARY KEY (proposalNumber, principalInvestigator, collaborators)
);

DROP TRIGGER IF EXISTS check_collaborators;
CREATE TRIGGER check_collaborators
BEFORE INSERT ON GrantProposals
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN NEW.collaborators = NEW.principalInvestigator THEN
            RAISE(ABORT, 'Collaborators cannot be the same as Principal Investigator')
    END;
END;

 * sqlite:///council.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [6]:
%%sql
DROP TABLE IF EXISTS ReviewerDetails;
CREATE TABLE ReviewerDetails (
    reviewer TEXT,
    proposalNumber INTEGER,
    coReviewers TEXT DEFAULT '',
    FOREIGN KEY (reviewer) REFERENCES Researcher(emailAddress),
    FOREIGN KEY (proposalNumber) REFERENCES ProposalDetails(proposalNumber),
    FOREIGN KEY (coReviewers) REFERENCES Researcher(emailAddress),
    PRIMARY KEY (reviewer, proposalNumber, coReviewers)
);

DROP TRIGGER IF EXISTS check_coReviewers;
CREATE TRIGGER check_coReviewers
BEFORE INSERT ON ReviewerDetails
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN NEW.reviewer = NEW.coReviewers THEN
            RAISE(ABORT, 'reviewer and coReviewers cannot have the same value')
    END;
END;

 * sqlite:///council.db
Done.
Done.
Done.
Done.


[]

In [7]:
%%sql
DROP TABLE IF EXISTS ReviewerConflicts;
CREATE TABLE ReviewerConflicts (
    reviewer TEXT,
    conflicts TEXT,
    FOREIGN KEY (reviewer) REFERENCES ReviewerDetails(reviewer),
    PRIMARY KEY (reviewer, conflicts) 
);

DROP TRIGGER IF EXISTS check_conflicts;
CREATE TRIGGER check_conflicts
BEFORE INSERT ON ReviewerConflicts
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN NEW.reviewer = NEW.conflicts THEN
            RAISE(ABORT, 'reviewer and coReviewers cannot have the same value')
    END;
END;

 * sqlite:///council.db
Done.
Done.
Done.
Done.


[]

In [8]:
%%sql
DROP TABLE IF EXISTS ReviewerAssignment;
CREATE TABLE ReviewerAssignment (
    reviewerAssignment INTEGER PRIMARY KEY AUTOINCREMENT,
    grantCompetition INTEGER,
    reviewers TEXT,
    deadline DATE DEFAULT CURRENT_DATE,
    status TEXT NOT NULL DEFAULT 'not submitted' CHECK (status IN ('submitted', 'not submitted')),
    FOREIGN KEY (grantCompetition) REFERENCES Grants(callNumber),
    FOREIGN KEY (reviewers) REFERENCES ReviewerDetails(reviewer)
);

 * sqlite:///council.db
Done.
Done.


[]

In [9]:
%%sql
DROP TABLE IF EXISTS GrantMeeting;
CREATE TABLE GrantMeeting (
    proposalNumber INTEGER,
    callNumber INTEGER,
    reviewers TEXT,
    date DATE,
    PRIMARY KEY (proposalNumber, callNumber),
    FOREIGN KEY (callNumber, reviewers) REFERENCES ReviewerAssignment(grantCompetition, reviewers)
);

 * sqlite:///council.db
Done.
Done.


[]

In [10]:
%%sql
INSERT INTO Researcher (emailAddress, firstName, lastName, organization) VALUES
('john@example.com', 'John', 'Doe', 'University A'),
('alice@example.com', 'Alice', 'Smith', 'University B'),
('bob@example.com', 'Bob', 'Johnson', 'University C'),
('emma@example.com', 'Emma', 'Davis', 'University D'),
('mike@example.com', 'Mike', 'Brown', 'University E'),
('sarah@example.com', 'Sarah', 'Wilson', 'University F'),
('chris@example.com', 'Chris', 'Jones', 'University G'),
('jessica@example.com', 'Jessica', 'Taylor', 'University H'),
('ryan@example.com', 'Ryan', 'Clark', 'University I'),
('lisa@example.com', 'Lisa', 'Martinez', 'University J');

INSERT INTO Grants (callNumber, status) VALUES
(101, 'open'),
(102, 'closed'),
(103, 'open'),
(104, 'open'),
(105, 'closed'),
(106, 'open'),
(107, 'closed'),
(108, 'open'),
(109, 'closed'),
(110, 'open');

INSERT INTO GrantDetails (callNumber, title, description, area) VALUES
(101, 'Research Grant', 'Funding for research projects', 'Science'),
(102, 'Education Grant', 'Funding for educational initiatives', 'Education'),
(103, 'Healthcare Grant', 'Funding for healthcare projects', 'Health'),
(104, 'Community Grant', 'Funding for community development', 'Community'),
(105, 'Technology Grant', 'Funding for technology projects', 'Technology'),
(106, 'Arts Grant', 'Funding for arts and culture', 'Arts'),
(107, 'Environment Grant', 'Funding for environmental projects', 'Environment'),
(108, 'Sports Grant', 'Funding for sports and recreation', 'Sports'),
(109, 'Business Grant', 'Funding for business initiatives', 'Business'),
(110, 'Humanitarian Grant', 'Funding for humanitarian projects', 'Humanitarian');


INSERT INTO ProposalDetails (proposalNumber, requestedAmount, applicationStatus, awardedAmount, date) VALUES
(1, 25000, 'submitted', NULL, '2024-03-15'),
(2, 15000, 'awarded', 12000, '2024-03-20'),
(3, 30000, 'not awarded', NULL, '2024-03-10'),
(4, 20000, 'submitted', NULL, '2024-03-12'),
(5, 35000, 'submitted', NULL, '2024-03-18'),
(6, 18000, 'submitted', NULL, '2024-03-22'),
(7, 28000, 'awarded', 24000, '2024-03-25'),
(8, 32000, 'not awarded', NULL, '2024-03-30'),
(9, 21000, 'submitted', NULL, '2024-03-11'),
(10, 40000, 'awarded', 35000, '2024-03-19');

 * sqlite:///council.db
10 rows affected.
10 rows affected.
10 rows affected.
10 rows affected.


[]

In [11]:
%%sql
INSERT INTO GrantProposals (proposalNumber, callNumber, principalInvestigator, collaborators) VALUES
(2, 102, 'bob@example.com', 'mike@example.com'),
(2, 102, 'bob@example.com', 'emma@example.com'),
(3, 103, 'mike@example.com', 'alice@example.com'),
(4, 104, 'mike@example.com', 'john@example.com'),
(5, 107, 'chris@example.com', 'alice@example.com'),
(6, 107, 'chris@example.com', 'emma@example.com'),
(7, 108, 'ryan@example.com', 'alice@example.com'),
(8, 110, 'lisa@example.com', 'john@example.com');

 * sqlite:///council.db
8 rows affected.


[]

In [12]:
%%sql
INSERT INTO ReviewerDetails (reviewer, proposalNumber, coReviewers) VALUES
('john@example.com', 1, 'alice@example.com'),
('john@example.com', 1, 'lisa@example.com'),
('bob@example.com', 3, 'mike@example.com'),
('bob@example.com', 3, 'john@example.com'),
('mike@example.com', 5, 'alice@example.com'),
('sarah@example.com', 6, 'john@example.com'),
('chris@example.com', 7, 'alice@example.com'),
('jessica@example.com', 7, 'emma@example.com'),
('ryan@example.com', 9, 'alice@example.com'),
('lisa@example.com', 10, 'john@example.com');


INSERT INTO ReviewerConflicts (reviewer, conflicts) VALUES
('john@example.com', 'alice@example.com'),
('john@example.com', 'bob@example.com'),
('bob@example.com', 'mike@example.com'),
('emma@example.com', 'bob@example.com'),
('mike@example.com', 'emma@example.com'),
('mike@example.com', 'john@example.com'),
('chris@example.com', 'alice@example.com'),
('jessica@example.com', 'bob@example.com'),
('ryan@example.com', 'alice@example.com'),
('ryan@example.com', 'jessica@example.com');

 * sqlite:///council.db
10 rows affected.
10 rows affected.


[]

In [13]:
%%sql
INSERT INTO ReviewerAssignment (grantCompetition, reviewers, deadline, status) VALUES
(101, 'john@example.com', '2024-03-25', 'submitted'),
(101, 'alice@example.com', '2024-03-25', 'submitted'),
(102, 'bob@example.com', '2024-03-28', 'submitted'),
(103, 'emma@example.com', '2024-03-30', 'submitted'),
(104, 'mike@example.com', '2024-04-02', 'submitted'),
(105, 'sarah@example.com', '2024-04-05', 'submitted'),
(106, 'chris@example.com', '2024-04-08', 'submitted'),
(107, 'jessica@example.com', '2024-04-10', 'submitted'),
(108, 'ryan@example.com', '2024-04-12', 'submitted'),
(109, 'lisa@example.com', '2024-04-15', 'submitted');

INSERT INTO GrantMeeting (proposalNumber, callNumber, reviewers, date) VALUES
(1, 101, 'john@example.com, alice@example.com', '2024-04-01'),
(2, 101, 'alice@example.com, bob@example.com', '2024-04-01'),
(3, 102, 'bob@example.com, emma@example.com', '2024-04-02'),
(4, 103, 'emma@example.com, mike@example.com', '2024-04-03'),
(5, 104, 'mike@example.com, sarah@example.com', '2024-04-04'),
(6, 105, 'sarah@example.com, chris@example.com', '2024-04-05'),
(7, 106, 'chris@example.com, jessica@example.com', '2024-04-06'),
(8, 107, 'jessica@example.com, ryan@example.com', '2024-04-07'),
(9, 108, 'ryan@example.com, lisa@example.com', '2024-04-08'),
(10, 109, 'lisa@example.com, john@example.com', '2024-04-09');


 * sqlite:///council.db
10 rows affected.
10 rows affected.


[]